# Context Managers in Python

Many resources that your programs interact with have rules about how they are to be properly "opened" and "closed", wehre those words loosely represent the set-up and tear-down phases of the interaction.





## Summary



## References

* [`contextlib` — Utilities for with-statement contexts](https://docs.python.org/3/library/contextlib.html)
